In [132]:
import numpy as np
from collections import defaultdict
from sympy import *
init_printing()

In [5]:
dataraw = """\
19, 13, 30 @ -2,  1, -2
18, 19, 22 @ -1, -1, -2
20, 25, 34 @ -2, -2, -4
12, 31, 28 @ -1, -2, -1
20, 19, 15 @  1, -5, -3
"""

In [42]:
with open("input.txt") as f:
    dataraw = f.read()

In [53]:
R, V = [], []
for dat in dataraw.split("\n")[:-1]:
    r, v = dat.split(' @ ')
    r = list(map(int, r.split(',')))
    v = list(map(int, v.split(',')))
    R.append(r), V.append(v)
R = np.array(R)
V = np.array(V)
VX, VY, VZ = V.T
X, Y, X = R.T
#R, V, X, Y, VX, VY

## question 1

In [45]:
A = V[:,1]/V[:,0]
B = Y - A * X

In [47]:
def in_past(indices):
    for k in indices:
        if intersect_x < X[k] and VX[k]>0 or intersect_x > X[k] and VX[k] < 0:
            #print("in the past")
            return True
    return False

In [52]:
n_intersects = 0
#l1, l2 = 7, 27
l1, l2 = 200000000000000, 400000000000000
for i in range(len(R)):
    for j in range(i+1, len(R)):
        dA = A[i]-A[j]
        if dA==0:
            continue
        intersect_x = (B[j]-B[i]) / dA
        intersect_y = A[i]*intersect_x + B[i]
        # check in the past:
        if in_past((i,j)):
            continue

        #print(f"{i}, {j}, {intersect_x:6.2f}, {intersect_y:6.2f}")
        if l1 < intersect_x and intersect_x < l2 and l1 < intersect_y and intersect_y < l2:
            #print("cross within window", i,j, X[i], X[j])
            n_intersects+=1
n_intersects

12740

## question 2

In [114]:
# initial try: search for parallel lines: dy/dx, dy/dz & dz/dx
ps = defaultdict(list)

for d1, d2 in ((0,1), (1,2), (0,2)):
    A = V[:,d2]/V[:,d1]
    for i in range(len(R)):
        for j in range(i+1, len(R)):
            dA = A[i]-A[j]
            if dA==0:
                ps[(d1,d2)].append([R[i], R[j], V[i], V[j]])
ps[(0,1)]

[[array([184506988243064, 319979732976203,  96196226089497]),
  array([225100803509611, 311851533614431, 264224583243849]),
  array([ 42,  24, 677]),
  array([-21, -12, -11])],
 [array([316185883798270, 360629781569228, 240116441670942]),
  array([411365066171366, 192855150652535, 290624821968082]),
  array([-116, -174,   65]),
  array([-12, -18,  16])],
 [array([374598513905778, 232760497993333, 485849701123497]),
  array([165481005413394, 339816397288489, 262440082454706]),
  array([-136,   34, -343]),
  array([ 80, -20, -64])]]

In [109]:
P = []
for i in (0,1):
    points = ps[(0,1)][i][2:4] # point line1, points line2, points line1+V
    points.append(ps[(0,1)][0][2] + 100000000*ps[(0,1)][i][4])
    P.append(points)
P

[[array([184506988243064, 319979732976203,  96196226089497]),
  array([225100803509611, 311851533614431, 264224583243849]),
  array([184511188243064, 319982132976203,  96263926089497])],
 [array([316185883798270, 360629781569228, 240116441670942]),
  array([411365066171366, 192855150652535, 290624821968082]),
  array([184495388243064, 319962332976203,  96202726089497])]]

In [110]:
line_intersection(*P)

/Users/teunissen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: RuntimeWarning: overflow encountered in long_scalars
/Users/teunissen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: RuntimeWarning: divide by zero encountered in long_scalars


(array([-5716004388043630080, -5973018201745631232, -4107857904242179328]),
 array([inf, inf, inf]))

In [98]:
points1

(array([184506988243064, 319979732976203,  96196226089497]),
 array([225100803509611, 311851533614431, 264224583243849]))

## question 2 other

In [140]:
# the new equation = v*t + r0 should equal the equations of the hailstones so:
# vt + r0 - (vv*t + rr0)
x,y,z,vx,vy,vz = symbols("x y z vx vy vz")
r = [x,y,z]
v = [vx,vy,vz]
var = [x,y,z,vx,vy,vz]
eqs = set()
for i, (rr, vv) in enumerate(zip(R[:3],V[:3])):
    print(rr,vv)
    t = Symbol(f"t_{i}")
    for j in range(3):
        eqs.add((v[j]*t + r[j]) - (vv[j]*t + rr[j]))
    var.append(t)
print(var)
sol = solve_poly_system(eqs, *var)
sol

[152594199160345 147562599184759 291883234654893] [229 220 -31]
[181402578613976 206158696386036 294595238970734] [179  99 -32]
[306345582484815 290719456201785 306246299945991] [-19 -64 -43]
[x, y, z, vx, vy, vz, t_0, t_1, t_2]


In [141]:
int(sum(sol[0][:3]))

In [136]:
eqs